In [8]:
# riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
import time 

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.")
print("=" *100)
query_txt = input('1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): ')
print(" 2. 위 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?")
print("1.학위논문", " "*8, "2.국내학술논문", " "*6, "3.해외학술논문", " "*6 ,"4.학술지")
print("5.단행본", " "*10, "6.공개강의", " "*10, "7.연구보고서")
info = {1:"학위논문", 2:"국내학술논문", 3:"해외학술논문", 4:"학술지", 5:"단행본", 6:"공개강의", 7:"연구보고서"}
query_txt2 = input('위 장르 중 수집할 장르의 번호를 입력하세요: ')
query_txt3 = info[int(query_txt2)]

#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 
fc_name = input('3.결과를 저장할 csv형식의 파일명을 쓰세요(예: c:\\temp\\riss.csv): ')
fx_name = input('4.결과를 저장할 xls형식의 파일명을 쓰세요(예: c:\\temp\\riss.xls): ')

#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
chrome_path = "C:/data/chromedriver"
driver = webdriver.Chrome(chrome_path)

url = 'http://www.riss.kr/'
driver.get(url)
time.sleep(2)

#Step 5. 자동으로 검색어 입력 후 조회하기
element = driver.find_element_by_id("query")
driver.find_element_by_id("query").click( )
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(2)

#Step 6.학위 논문 선택하기
driver.find_element_by_link_text(query_txt3).click()
time.sleep(2)

#Step 7.Beautiful Soup 로 본문 내용만 추출하기
from bs4 import BeautifulSoup
html_1 = driver.page_source
soup_1 = BeautifulSoup(html_1, 'html.parser')

content_1 = soup_1.find('div','srchResultListW').find_all('li')
for i in content_1 :
    print(i.get_text().replace("\n",""))

#Step 8. 총 검색 건수를 보여주고 수집할 건수 입력받기
import math
total_cnt = soup_1.find('div','searchBox').find('span','num').get_text()
print('검색하신 키워드 %s (으)로 총 %s 건의 학위논문이 검색되었습니다' %(query_txt,total_cnt))
collect_cnt = int(input('이 중에서 몇 건을 수집하시겠습니까?: '))
collect_page_cnt = math.ceil(collect_cnt / 10)
print('%s 건의 데이터를 수집하기 위해 %s 페이지의 게시물을 조회합니다.' %(collect_cnt,collect_page_cnt))

#Step 9. 각 항목별로 데이터를 추출하여 리스트에 저장하기
no2 = [ ]        #번호 저장
title2 = [ ]     #논문제목 저장
writer2 = [ ]    #논문저자 저장
org2 = [ ]       #소속기관 저장
no = 1
a1 = [ ] # 발표년도
b1 = [ ] # 논문집/자료집
c1 = [ ] # 논문URL정보

# 다음 페이지 번호 만들기
page_no=[ ]

for i in range(10,collect_cnt) :
    if i % 10 == 0 :
            page_no.append(i + 1)

for a in range(1, collect_page_cnt + 1) :
    
    html_2 = driver.page_source
    soup_2 = BeautifulSoup(html_2, 'html.parser')

    content_2 = soup_2.find('div','srchResultListW').find_all('li')
    
    for b in content_2 :    
        #1. 논문제목 있을 경우만
        try :
            title = b.find('div','cont').find('p','title').get_text()
        except :
            continue
        else :
            
            print('1.번호:',no)
            no2.append(no)
            

            print('2.논문제목:',title)
            title2.append(title)
            
            
            writer = b.find('span','writer').get_text()
            print('3.저자:',writer)
            writer2.append(writer)
            

            org = b.find('span','assigned').get_text()
            print('4.소속기관:' , org)
            org2.append(org)
            

            a2 = b.select_one('div.cont > p.etc > span:nth-child(3)').get_text()
            print('5.발표년도:' , a2)
            a1.append(a2)
            
            b2 = b.find('a').get_text()
            print('6.논문집/자료집:' , b2)
            b1.append(b2)


            c2 = b.find('a')["href"]
            print('7.논문 URL정보:' , c2)
            c1.append(c2)
            
            
            
            
            no += 1
            print("\n")
            
            if no > collect_cnt :
                break

            time.sleep(1)        # 페이지 변경 전 1초 대기 

        c = math.floor(a/10)-1 
        a += 1
        
        try:
            if a == page_no[c]:
                driver.find_element_by_link_text('다음 페이지로').click()
            else :
                driver.find_element_by_link_text('%s' %a).click() # 다음 페이지번호 클릭

        except:
                print("요청하신 작업이 모두 완료되었습니다")

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
import pandas as pd 

df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['작성자']=pd.Series(writer2)
df['소속기관']=pd.Series(org2)
df['발표년도']=pd.Series(a1)
df['논문집/자료집']=pd.Series(b1)
df['논문 URL정보']=pd.Series(c1)


# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8")

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')


 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.
1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): 해양자원
 2. 위 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?
1.학위논문          2.국내학술논문        3.해외학술논문        4.학술지
5.단행본            6.공개강의            7.연구보고서
위 장르 중 수집할 장르의 번호를 입력하세요: 2
3.결과를 저장할 csv형식의 파일명을 쓰세요(예: c:\temp\riss.csv): C:/data/riss2.csv
4.결과를 저장할 xls형식의 파일명을 쓰세요(예: c:\temp\riss.xls): C:/data/riss3.xls
1발표논문 / 해양생물자원으로서 해조류 : 생물활성물질의 정제와 분자적 응용홍용기(Yong Ki Hong)한국조류학회2000국제심포지움 일정 및 발표논문집 - 21세기, 해양환경과 해양생물자원의 전망Vol.- No.-해조생물공학은 유용해조류 및 그들의 유용물질들을 경제적으로 생산할 수 있도록 개발하는 학문 분야이다. 본 논문에서는 이들 해조생물공학에 대한 실험방법적 접근을 강조하여 설명하고자 한다. 우선 해조류의 활력측정은 조직 자체의 효소활성 즉 무색의 2,3,5-triphenyltetrazolium chloride를 붉은색의 triphenylformazan으로 환원시키는 정도를 측정함으로서 정량적으로 간단하게 활력을 평가하는 방법을 확립하였다. 그리고 callus와 엽체 재생 등을 위한 무균 고체 조직배양에 가장 적합한 gelling agent들을 비교한바 high gel strength agar가 톳의 callus 형성 및 엽체 재생에 지지체로 가장 효율적이였다. 유용 생리활성물질의 탐색작업에서는 보라색우무 추출물에서 Taq DNA polymerase에 대한 저해작용을 보였으며, 청각, 구멍갈파래, 잎파래, 미역에는 Herpes simplex virus에 대한 항virus효과를 

이 중에서 몇 건을 수집하시겠습니까?: 30
30 건의 데이터를 수집하기 위해 3 페이지의 게시물을 조회합니다.
1.번호: 1
2.논문제목: 발표논문 / 해양생물자원으로서 해조류 : 생물활성물질의 정제와 분자적 응용
3.저자: 홍용기(Yong Ki Hong)
4.소속기관: 한국조류학회
5.발표년도: 2000
6.논문집/자료집: 발표논문 / 해양생물자원으로서 해조류 : 생물활성물질의 정제와 분자적 응용
7.논문 URL정보: /search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=508c47ca1906d5ecffe0bdc3ef48d419


1.번호: 2
2.논문제목: 수거된 해양폐기물 자원화 기술 개발(Ⅰ)
3.저자: 길상인(Sang-In Keel), 윤진한(Jin-Han Yun), 최연석(Yeon-Seok Choi), 강창구(Chang-Gu Kang), 유정석(Jeong-Seok Yu)
4.소속기관: 한국해양환경·에너지학회
5.발표년도: 2002
6.논문집/자료집: 수거된 해양폐기물 자원화 기술 개발(Ⅰ)
7.논문 URL정보: /search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=09ec1e3541fcc118ffe0bdc3ef48d419


1.번호: 3
2.논문제목: 해양경찰 미래 발전을 위한 인적 자원 확보 방안연구
3.저자: 윤병두
4.소속기관: 한국해양경찰학회
5.발표년도: 2021
6.논문집/자료집: 해양경찰 미래 발전을 위한 인적 자원 확보 방안연구
7.논문 URL정보: /search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=fcc6dea7df1a2c0fd18150b21a227875


1.번호: 4
2.논문제목: 해양관광 활성화를 위한 해양문화관광자원 활용 방안 해양문화축제를 중심으로
3.저자: 하경희(Ha, Kyoung-Hee)
4.소속기관: 한국해

요청하신 작업이 모두 완료되었습니다
1.번호: 30
2.논문제목: 폐기물 해양배출에 대한 해양환경적 고찰
3.저자: 박용철,최중기
4.소속기관: 한국해양학회
5.발표년도: 1993
6.논문집/자료집: 폐기물 해양배출에 대한 해양환경적 고찰
7.논문 URL정보: /search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=7d257b88b6c7359d4884a65323211ff0


요청하신 데이터 수집 작업이 정상적으로 완료되었습니다


<ipython-input-8-3dd4ab99edd9>:159: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel(fx_name,index=False, encoding="utf-8")


In [6]:
query_txt2

'2'

In [9]:
info

{1: '학위논문',
 2: '국내학술논문',
 3: '해외학술논문',
 4: '학술지',
 5: '단행본',
 6: '공개강의',
 7: '연구보고서'}

In [11]:
info[1]

'학위논문'

In [ ]:
C:/data/riss2.csv

In [ ]:
            b2 = b.find('span','assigned').get_text()
            print('6.논문집/자료집:' , b2)
            b1.append(org)
            